<a href="https://colab.research.google.com/github/shravan1994/disaster_prediction_using_tweets/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.metrics import f1_score
import tensorflow_hub as hub

In [ ]:
!pip install contractions
!pip install -q -U "tensorflow-text==2.8.*"
!pip install bert-tensorflow
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 106 kB 4.9 MB/s 
     |████████████████████████████████| 287 kB 14.0 MB/s 
     |████████████████████████████████| 4.9 MB 5.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 64 kB 2.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import Input, Dropout, Dense, Embedding, Conv1D, MaxPool1D, Flatten
from tensorflow.keras import Model
from bert.tokenization import FullTokenizer
from tensorflow.keras.callbacks import Callback
import urllib
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import hstack
import tensorflow_text as text

## Model Architecture

In [ ]:
bert_preprocess_model = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4', trainable=True)

In [ ]:
def get_model_architecture():
  input_layer1 = Input(shape=(), dtype=tf.string, name='text-layer')
  text_preprocessed = bert_preprocess_model(input_layer1)
  bert_outputs = bert_layer(text_preprocessed)
  pooled_out = bert_outputs['pooled_output']
  dropout_1 = Dropout(0.1)(pooled_out)
  bert_out = tf.keras.layers.Reshape((32,24))(dropout_1)
      
  lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(bert_out)
  dropout_3 = tf.keras.layers.Dropout(0.1)(lstm)
  cnn = tf.keras.layers.Conv1D(64,5)(dropout_3)

  max_pool = tf.keras.layers.MaxPooling1D(25)(cnn)
  flat = tf.keras.layers.Flatten(data_format='channels_last')(max_pool)

  dense1 = tf.keras.layers.Dense(128,activation='relu')(pooled_out)
  dense2 = tf.keras.layers.Dense(64,activation='relu')(dense1)
  dropout_2 = tf.keras.layers.Dropout(0.1)(dense2)

  concat = tf.keras.layers.Concatenate()([dropout_2,flat])

  output = Dense(1,activation='sigmoid')(concat)

  model_bert_cnn = Model(inputs=input_layer1, outputs=output)
  return model_bert_cnn

## Methods for data preprocessing

In [ ]:
import re

def replace_byte_chars(tweet):
  tweet = re.sub(r"\x89Ûªs", '\'s', tweet)
  tweet = re.sub(r"\x89Û_", "", tweet)
  tweet = re.sub(r"\x89ÛÒ", "", tweet)
  tweet = re.sub(r"\x89ûó", "", tweet)
  tweet = re.sub(r"\x89ÛÏ", "", tweet)
  tweet = re.sub(r"\x89Û÷", "", tweet)
  tweet = re.sub(r"\x89Û", "", tweet)
  tweet = re.sub(r"\x89Û\x9d", "", tweet)
  tweet = re.sub(r"\x89Û¢", "", tweet)
  tweet = re.sub(r"\x89Û¢åÊ", "", tweet)
  tweet = re.sub(r"åÊ", " ", tweet)
  tweet = re.sub(r"fromåÊwounds", "from wounds", tweet)
  tweet = re.sub(r"JapÌ_n", "Japan", tweet)    
  tweet = re.sub(r"Ì©", "e", tweet)
  tweet = re.sub(r"å¨", "", tweet)
  tweet = re.sub(r"SuruÌ¤", "Suruc", tweet)
  tweet = re.sub(r"åÇ", "", tweet)
  tweet = re.sub(r"å£3million", "3 million", tweet)
  tweet = re.sub(r"åÀ", "", tweet)
  
  return tweet

In [ ]:
import contractions
def do_decontractions(tweet):
  tweet = contractions.fix(tweet)
  return tweet

In [ ]:
def clean_tweet(tweet):
  tweet = tweet.lower()
  tweet = replace_byte_chars(tweet)
  tweet = do_decontractions(tweet)
  # removing urls
  tweet = re.sub(r"https?:\/\/t.co\/[A-Za-z0-9]+", "", tweet)
  # Words with punctuations and special characters
  tweet = re.sub(r"[\"#$%&'()*+,\-.\/:;<=>@[\]^_`{|}~]", "", tweet)
  # adding space in front of ? and !
  tweet = re.sub(r"([?!]+)", r" \1", tweet)
  tweet = re.sub(r"\s+", " ", tweet)
  tweet = tweet.strip()
  return tweet

## final_func_1(), this method returns predicted target labels

In [ ]:
def final_fun_1(X):
  model_path = '/content/drive/MyDrive/Colab Notebooks/Self case study/Natutal Disaster prediction/model/best_bert_cnn_model.h5'

  if type(X) == str:
    X = pd.Series([X]) 
  
  X = X.apply(lambda x: clean_tweet(x))
  
  model = get_model_architecture()
  model.load_weights(model_path)

  target_predicted = model.predict(X)
  target_predicted = [1 if target > 0.5 else 0 for target in target_predicted]

  return target_predicted

## final_func_2(), this method returns log loss

In [ ]:
def final_fun_2(X, y):
  target_predicted = final_fun_1(X)
  f1 = f1_score(y, target_predicted, average='macro')
  return f1

## Testing above two functions

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "."

!kaggle competitions download -c nlp-getting-started
!unzip nlp-getting-started.zip -d 'data'

  0% 0.00/593k [00:00<?, ?B/s]
100% 593k/593k [00:00<00:00, 102MB/s]
Archive:  nlp-getting-started.zip
  inflating: data/sample_submission.csv  
  inflating: data/test.csv           
  inflating: data/train.csv          


### Testing func 1

In [ ]:
X_df = pd.read_csv('data/test.csv')
predicted_targets = final_fun_1(X_df['text'])
predicted_targets[0:20]

[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0]

### Testing func 2

In [ ]:
X_df = pd.read_csv('data/train.csv')
f1 = final_fun_2(X_df['text'], X_df['target'])
print('F1 score: ', f1)

F1 score:  0.8500380099963321
